In [7]:
# bob.refresh {"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}
# bob.static {"token": "4622c4cc3822f6c1164cadbd53ed0d1230fb0a562d7c6bc824116cd82a78", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_HOSP_DEP_NAME.html", "filename": "../data_output/France_HOSP_DEP_NAME.html"}
# bob.static {"token": "24e8978d3f09f857f81ed59d08c2b01a98e7eb0a95b85d985158b449bf0a", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_REA_DEP_NAME.html", "filename": "../data_output/France_REA_DEP_NAME.html"} 
# bob.static {"token": "83596ba6c97aa043e1994c93e4fc49354cd31842c2f83d66da0ce9dc314f", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_DC_DEP_NAME.html", "filename": "../data_output/France_DC_DEP_NAME.html"} 
# bob.static {"token": "09e1f0c28332e7b570cb13fe66def2187a457697a9a54921c96cfb2f76fd", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_RAD_DEP_NAME.html", "filename": "../data_output/France_RAD_DEP_NAME.html"} 

# bob.static {"token": "113a1888cfa76275418845a0a21fbed1e2f766bf12a0a7b9b72c20de1aa5", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_HOSP_REG_NAME.html", "filename": "../data_output/France_HOSP_REG_NAME.html"}
# bob.static {"token": "1bf05f448efec2591a9b59b2a8683b24eb99eaf3314f7a6d87937d05f7f4", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_REA_REG_NAME.html", "filename": "../data_output/France_REA_REG_NAME.html"}
# bob.static {"token": "a1e16c779ba48901c40ad7daf39dceea2dda8752b6de257b58108a97b9eb", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_DC_REG_NAME.html", "filename": "../data_output/France_DC_REG_NAME.html"} 
# bob.static {"token": "5fd5e5d8525afa27210b5c64420d475a531c3b1eb3d6bbd9fed979f3081d", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_RAD_REG_NAME.html", "filename": "../data_output/France_RAD_REG_NAME.html"} 

# bob.static {"token": "3a0e8e24ed4c4a9631007d76a988bd40660844e2e6ce6a631a528c22774d", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_TSHOSP_DEP_NAME.html", "filename": "../data_output/France_TSHOSP_DEP_NAME.html"}
# bob.static {"token": "ec820131eb62001106cb3f0a74f2adbacefb68f5d11854be593972e4f55f", "path": "/home/ftp/ftp/dev/wsr-public/data_output/France_TSHOSP_REG_NAME.html", "filename": "../data_output/France_TSHOSP_REG_NAME.html"}

## Database
*italic* <br>
**gras**
* Point 1
* Point 2

In [8]:
%run __init__.ipynb

Successfully connected to MongoDB


In [9]:
def analysis(df,variable):
    Min = df[variable].min()
    Max = df[variable].max()
    Avr = round(df[variable].mean(),0)
    Med = round(np.median(df[variable]),0)
    AvrMax = round((Avr / Max),2)
    MedMax = round((Med / Max),2)
    to_print = f'{variable} : Min: {Min}, Max: {Max}, Average: {Avr}, AvrMax: {AvrMax}, Med: {Med}, MedMax: {MedMax}'
    return print(to_print)

In [10]:
#Var France
map_center = {"lat": 46.7667, "lon": 2.45}
def create_map(df,geo,dfKey,geoKey,scale,colors,map_center,label,file_html):
    fig = px.choropleth_mapbox(df, 
                               geojson=geo, 
                               color="VALUE",
                               locations=dfKey, 
                               featureidkey=geoKey,
                               range_color=scale,
                               color_continuous_scale=colors,
                               mapbox_style="carto-positron", 
                               zoom=5, 
                               center=map_center,
                               hover_data=["VALUE","VAR"],
                               labels=label,
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True)
    fig.write_html(f"{output_folder}{file_html}")
    updateChartCss(f"{output_folder}{file_html}", f"{input_folder}map.css")
#     return fig

In [11]:
#--- Check if the data are already updated
filePath = f'{output_folder}107_FRANCE.csv'
if path.exists(filePath):
    df_check = pd.read_csv(filePath, sep=';',low_memory=False)
    date_check = df_check[df_check['SCENARIO'] == yesterday.strftime('%d/%m/%Y')]
    if len(date_check) > 0:
        print(f'Data already updated ! DateTime : {now}')
        %stop

In [12]:
#Source 107
df = pd.read_csv(output_folder + '107.csv',sep=";").drop('Unnamed: 0',axis=1)
df = df[(df['SCENARIO'] == yesterday.strftime('%d/%m/%Y'))].reset_index(drop=True)
if len(df) == 0:
    print(f'Data not updated ! DateTime : {now}')
    %stop
else:
    df.to_csv(output_folder + '107_FRANCE.csv',sep=";")

df = pd.merge(df,ref_france, left_on='CHILD', right_on='DEP_FULL',how='left').rename(index=str, columns={"DEP_NAME":"LABEL"}).drop(['DEP_CODE','DEP_FULL','REG_NAME'], axis=1)
df.loc[df['PARENT'] == 'root', 'LABEL'] = df['CHILD']
df = df.drop(['UNIT_VARP','PRECISION_VALUE','PRECISION_VAR','PRECISION_VARP','LAST_UPDATE'],axis=1)
df.head(5)

,ENTITY,SCENARIO,DATE_ORDER,CHILD,PARENT,UPPER_FILTER_M,ORDER,VALUE,VAR,VARP,LABEL
0,France,03/05/2020,20200503,Île-de-France,root,En réanimation,2.0,1550,-2,-0.128866,Île-de-France
1,France,03/05/2020,20200503,Île-de-France,root,Hospitalisations,1.0,10685,-30,-0.279981,Île-de-France
2,France,03/05/2020,20200503,31 - Haute-Garonne,Occitanie,Hospitalisations,1.0,125,3,2.459016,Haute-Garonne
3,France,03/05/2020,20200503,32 - Gers,Occitanie,Hospitalisations,1.0,28,1,3.703704,Gers
4,France,03/05/2020,20200503,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,1.0,212,5,2.415459,Gironde


In [13]:
#Hospitalisation - Département
kpi = 'Hospitalisations'
file_html = "France_HOSP_DEP_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,3000)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_dep,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 2, Max: 2177, Average: 255.0, AvrMax: 0.12, Med: 106.0, MedMax: 0.05


In [14]:
#En réanimation - Département
kpi = 'En réanimation'
file_html = "France_REA_DEP_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,800)
colors = [
    [0.0, "#FFFFFF"],
    [0.025,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_dep,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0, Max: 430, Average: 37.0, AvrMax: 0.09, Med: 13.0, MedMax: 0.03


In [15]:
#"Décès à l’hôpital" - Département
kpi = "Décès à l’hôpital"
file_html = "France_DC_DEP_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,1500)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_dep,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0, Max: 1493, Average: 154.0, AvrMax: 0.1, Med: 60.0, MedMax: 0.04


In [16]:
#'Retours à domicile' - Département
kpi = 'Retours à domicile'
file_html = "France_RAD_DEP_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] != 'root')]
analysis(df_map,'VALUE')

scale = (0,3000)
colors = [
    [0.0, "#FFFFFF"],
    [0.05,"#B3D7D2"],
    [0.10, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Département','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_dep,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 18, Max: 4229, Average: 503.0, AvrMax: 0.12, Med: 178.0, MedMax: 0.04


#ANALYSIS BY REGION

In [17]:
#Hospitalisation - Région
kpi = 'Hospitalisations'
file_html = "France_HOSP_REG_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] == 'root') & (df['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (100,13000)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_reg,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 2, Max: 10685, Average: 1430.0, AvrMax: 0.13, Med: 601.0, MedMax: 0.06


In [18]:
#En réanimation - Région
kpi = 'En réanimation'
file_html = "France_REA_REG_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] == 'root') & (df['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (50,2500)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.55, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_reg,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0, Max: 1550, Average: 209.0, AvrMax: 0.13, Med: 98.0, MedMax: 0.06


In [19]:
#"Décès à l’hôpital" - Région
kpi = "Décès à l’hôpital"
file_html = "France_DC_REG_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] == 'root') & (df['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (100,5500)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_reg,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 0, Max: 6116, Average: 865.0, AvrMax: 0.14, Med: 351.0, MedMax: 0.06


In [20]:
#Retours à domicile - Région
kpi = 'Retours à domicile'
file_html = "France_RAD_REG_NAME.html"

df_map = df.copy()[(df['UPPER_FILTER_M'] == kpi) & (df['PARENT'] == 'root') & (df['LABEL'] != 'France')]
analysis(df_map,'VALUE')

scale = (200,15000)
colors = [
    [0.0, "#FFFFFF"],
    [0.075,"#B3D7D2"],
    [0.15, "#81B1BD"],
    [0.4, "#437F9D"],
    [1.0, "#00354E"]
]
dfKey = "LABEL"
geoKey="properties.nom"
to_rename = {'LABEL':'Région','VALUE': kpi, 'VAR': 'Variation en nb.'}
create_map(df_map,france_reg,dfKey,geoKey,scale,colors,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")

VALUE : Min: 36, Max: 18056, Average: 2821.0, AvrMax: 0.16, Med: 1262.0, MedMax: 0.07


In [21]:
map_center = {"lat": 46.7667, "lon": 2.45}
def create_map_epid(df,geo,dfKey,geoKey,col_color,hoverdata,map_center,label,file_html):
    fig = px.choropleth_mapbox(df, 
                               geojson=geo, 
                               color=col_color,
                               locations=dfKey, 
                               featureidkey=geoKey,
                                color_discrete_map={ # replaces default color mapping by value
                                    "vert": "#4CCD88", "orange": "#FDB47A", "rouge": "#DD6D88"
                                },
                               mapbox_style="carto-positron", 
                               zoom=5, 
                               center=map_center,
                               hover_data= hoverdata,
                               labels=label,
                               opacity=0.8
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True, showlegend=False)
    fig.write_html(f"{output_folder}{file_html}")
    updateChartCss(f"{output_folder}{file_html}", f"{input_folder}map.css")
#     return fig

In [22]:
#--- Situation epidémiologique DATA
cols = ['DEP','DEP_NAME','REA_2013','REA_2018','SI_2013','SI_2018','SC_2013','SC_2018','ALL_2013','ALL_2018']
ref_france_lits = pd.read_excel(input_folder + 'drees_lits_reanimation_2013-2018.xlsx', sheet_name = 'Lits_2013-2018', header=7, names=cols)
ref_france_lits = ref_france_lits[['DEP','DEP_NAME','REA_2018']]
ref_france_lits['DEP'] = ref_france_lits['DEP'].astype(str)
ref_france_lits.head(5)

,DEP,DEP_NAME,REA_2018
0,FR,France entière,5432
1,MET,France métropolitaine,5290
2,01,Ain,22
3,02,Aisne,34
4,03,Allier,24


In [23]:
#---Tension Data
df_rea = pd.read_csv(output_folder + '107_FRANCE.csv',sep=";").drop('Unnamed: 0',axis=1)
df_rea = df_rea[['ENTITY','SCENARIO','CHILD','PARENT','UPPER_FILTER_M','VALUE','VAR']]
df_rea = pd.merge(df_rea,ref_france, left_on='CHILD', right_on='DEP_FULL',how='left').rename(index=str, columns={"DEP_NAME":"LABEL"}).drop(['DEP_FULL'], axis=1)
df_rea.loc[df_rea['PARENT'] == 'root', 'LABEL'] = df_rea['CHILD']
df_rea['DEP_CODE'] = df_rea['DEP_CODE'].astype(str)

df_rea.loc[df_rea['CHILD'] == 'France', 'DEP_CODE'] = 'FR'
df_rea = pd.merge(df_rea,ref_france_lits, left_on='DEP_CODE', right_on='DEP',how='left').drop(['DEP','DEP_NAME'], axis=1)
df_rea.head(5)

,ENTITY,SCENARIO,CHILD,PARENT,UPPER_FILTER_M,VALUE,VAR,DEP_CODE,LABEL,REG_NAME,REA_2018
0,France,03/05/2020,Île-de-France,root,En réanimation,1550,-2,nan,Île-de-France,NaN,NaN
1,France,03/05/2020,Île-de-France,root,Hospitalisations,10685,-30,nan,Île-de-France,NaN,NaN
2,France,03/05/2020,31 - Haute-Garonne,Occitanie,Hospitalisations,125,3,31,Haute-Garonne,Occitanie,165.0
3,France,03/05/2020,32 - Gers,Occitanie,Hospitalisations,28,1,32,Gers,Occitanie,8.0
4,France,03/05/2020,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,212,5,33,Gironde,Nouvelle-Aquitaine,195.0


In [24]:
#---HKPIS
df_hkpis = df_rea.copy()[(df_rea['CHILD'] == 'France') & (df_rea['SCENARIO'] == yesterday.strftime('%d/%m/%Y'))]
df_kpis = df_hkpis[['ENTITY','SCENARIO','UPPER_FILTER_M','VALUE','VAR']]

df_reakpi = df_hkpis.copy()[df_hkpis['UPPER_FILTER_M'] == 'En réanimation'][['ENTITY','SCENARIO','UPPER_FILTER_M','VALUE','VAR','REA_2018']]
df_reakpi['VALUEP'] = round((df_reakpi['VALUE'] / df_reakpi['REA_2018'] * 100),2)
df_reakpi['VALUE_N1'] = (df_reakpi['VALUE'] - df_reakpi['VAR'])
df_reakpi['VALUEP_N1'] = round(((df_reakpi['VALUE_N1'] / df_reakpi['REA_2018']) * 100),2)
df_reakpi['VALUEP_VAR'] = df_reakpi['VALUEP'] - df_reakpi['VALUEP_N1']

df_reakpi = df_reakpi.drop(['VALUE','VAR','REA_2018','VALUE_N1','VALUEP_N1'],axis=1).rename(index=str, columns={"VALUEP":"VALUE","VALUEP_VAR":"VAR"})
df_reakpi['UPPER_FILTER_M'] = 'Tensions hospitalières'

df_final = pd.concat([df_kpis,df_reakpi], axis=0)
df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','UNIT_VALUE'] = ' %'
df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','UNIT_VAR'] = ' pts'
df_final.loc[:,'PRECISION'] = ',.0f'
df_final.loc[df_final['UPPER_FILTER_M'] == 'Tensions hospitalières','PRECISION'] = ',.2f'

if use_mongo:
    bob.mongo.save_df(df_final,'110_BIS',db_app,True)
df_final

Dataframe 110_BIS successfully save in database app-wsr-dev in MongoDB. Time: --- 0.018590927124023438 secnds ---


,ENTITY,SCENARIO,UPPER_FILTER_M,VALUE,VAR,UNIT_VALUE,UNIT_VAR,PRECISION
77,France,03/05/2020,Hospitalisations,25739.00,-12.00,NaN,NaN,",.0f"
199,France,03/05/2020,Décès à l’hôpital,15564.00,96.00,NaN,NaN,",.0f"
316,France,03/05/2020,En réanimation,3762.00,-8.00,NaN,NaN,",.0f"
436,France,03/05/2020,Retours à domicile,50776.00,222.00,NaN,NaN,",.0f"
316,France,03/05/2020,Tensions hospitalières,69.26,-0.14,%,pts,",.2f"


In [25]:
#Par département
df_rea_dep = df_rea.copy()[(df_rea['UPPER_FILTER_M'] == 'En réanimation') & (df_rea['PARENT'] != 'root') & (df_rea['SCENARIO'] == yesterday.strftime('%d/%m/%Y'))]
df_rea_dep['VALUEP'] = round((df_rea_dep['VALUE'] / df_rea_dep['REA_2018'] * 100),2)
df_rea_dep.loc[df_rea_dep['VALUEP'] < 60 ,'COLOR'] = 'vert'
df_rea_dep.loc[df_rea_dep['VALUEP'] >= 60, 'COLOR'] = 'orange'
df_rea_dep.loc[df_rea_dep['VALUEP'] >= 80, 'COLOR'] = 'rouge'
df_rea_dep.head(5)

,ENTITY,SCENARIO,CHILD,PARENT,UPPER_FILTER_M,VALUE,VAR,DEP_CODE,LABEL,REG_NAME,REA_2018,VALUEP,COLOR
242,France,03/05/2020,01 - Ain,Auvergne-Rhône-Alpes,En réanimation,14,0,01,Ain,Auvergne-Rhône-Alpes,22.0,63.64,orange
243,France,03/05/2020,32 - Gers,Occitanie,En réanimation,4,0,32,Gers,Occitanie,8.0,50.00,vert
244,France,03/05/2020,33 - Gironde,Nouvelle-Aquitaine,En réanimation,45,0,33,Gironde,Nouvelle-Aquitaine,195.0,23.08,vert
245,France,03/05/2020,34 - Hérault,Occitanie,En réanimation,29,-1,34,Hérault,Occitanie,128.0,22.66,vert
246,France,03/05/2020,35 - Ille-et-Vilaine,Bretagne,En réanimation,14,0,35,Ille-et-Vilaine,Bretagne,66.0,21.21,vert


In [26]:
#---Tension_Département
kpi = 'Tension hospitalière (%)'
file_html = "France_TSHOSP_DEP_NAME.html"
dfKey = "LABEL"
geoKey="properties.nom"
col_color = 'COLOR'
hoverdata=["VALUE","VAR","REA_2018",'VALUEP']
to_rename = {'COLOR': "Niveau",'LABEL':'Département', 'VALUE': 'Nombre de cas en réanimation', 'VAR': 'Evolution en réanimation','REA_2018': 'Nombre de lits en service réanimation','VALUEP': kpi}
create_map_epid(df_rea_dep,france_dep,dfKey,geoKey,col_color,hoverdata,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")
# url ='https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/3a0e8e24ed4c4a9631007d76a988bd40660844e2e6ce6a631a528c22774d'

In [27]:
#Par région
df_rea_re = df_rea.copy()[(df_rea['UPPER_FILTER_M'] == 'En réanimation') & (df_rea['PARENT'] != 'root') & (df_rea['SCENARIO'] == yesterday.strftime('%d/%m/%Y'))]
cols_to_group = ['ENTITY','SCENARIO','REG_NAME']
df_rea_re['VALUE'] = df_rea_re['VALUE'].astype(int)
df_rea_re['VAR'] = df_rea_re['VAR'].astype(int)
df_rea_re = df_rea_re.groupby(cols_to_group, as_index=False).agg({"VALUE":'sum',"VAR":'sum',"REA_2018":'sum'})

df_rea_re['VALUEP'] = round((df_rea_re['VALUE'] / df_rea_re['REA_2018'] * 100),2)
df_rea_re.loc[df_rea_re['VALUEP'] < 60 ,'COLOR'] = 'vert'
df_rea_re.loc[df_rea_re['VALUEP'] >= 60, 'COLOR'] = 'orange'
df_rea_re.loc[df_rea_re['VALUEP'] >= 80, 'COLOR'] = 'rouge'
df_rea_re.head(5)

,ENTITY,SCENARIO,REG_NAME,VALUE,VAR,REA_2018,VALUEP,COLOR
0,France,03/05/2020,Auvergne-Rhône-Alpes,358,-1,609.0,58.78,vert
1,France,03/05/2020,Bourgogne-Franche-Comté,174,2,207.0,84.06,rouge
2,France,03/05/2020,Bretagne,45,0,171.0,26.32,vert
3,France,03/05/2020,Centre-Val de Loire,115,3,191.0,60.21,orange
4,France,03/05/2020,Corse,8,0,18.0,44.44,vert


In [28]:
#---Tension_Région
kpi = 'Tension hospitalière (%)'
file_html = "France_TSHOSP_REG_NAME.html"
dfKey = "REG_NAME"
geoKey="properties.nom"
col_color = 'COLOR'
hoverdata=["VALUE","VAR","REA_2018",'VALUEP']
to_rename = {'REG_NAME':'Région','COLOR': "Niveau", 'VALUE': 'Nombre de cas en réanimation','VAR': 'Evolution en réanimation', 'REA_2018': 'Nombre de lits en service réanimation','VALUEP': kpi}
create_map_epid(df_rea_re,france_reg,dfKey,geoKey,col_color,hoverdata,map_center,to_rename,file_html)
# bob.makeStaticGettable(output_folder + file_html,"dev")
# url ='https:/api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/ec820131eb62001106cb3f0a74f2adbacefb68f5d11854be593972e4f55f'

In [29]:
#html
#Par département
hosp_dep = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/4622c4cc3822f6c1164cadbd53ed0d1230fb0a562d7c6bc824116cd82a78', 'KPI' : 'Hospitalisations', 'UPPER_FILTER' : 'Par département'}]
rea_dep = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/24e8978d3f09f857f81ed59d08c2b01a98e7eb0a95b85d985158b449bf0a', 'KPI' : 'En réanimation', 'UPPER_FILTER' : 'Par département'}]
dc_dep = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/09e1f0c28332e7b570cb13fe66def2187a457697a9a54921c96cfb2f76fd', 'KPI' : 'Retours à domicile', 'UPPER_FILTER' : 'Par département'}]
rad_dep = [{'URL' :'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/83596ba6c97aa043e1994c93e4fc49354cd31842c2f83d66da0ce9dc314f', 'KPI' : 'Décès à l’hôpital', 'UPPER_FILTER' : 'Par département'}]
reat_dep = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/3a0e8e24ed4c4a9631007d76a988bd40660844e2e6ce6a631a528c22774d', 'KPI' : 'Tensions hospitalières', 'UPPER_FILTER' : 'Par département'}]
          
#Par région
hosp_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/113a1888cfa76275418845a0a21fbed1e2f766bf12a0a7b9b72c20de1aa5', 'KPI' : 'Hospitalisations', 'UPPER_FILTER' : 'Par région'}]
rea_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/1bf05f448efec2591a9b59b2a8683b24eb99eaf3314f7a6d87937d05f7f4', 'KPI' : 'En réanimation', 'UPPER_FILTER' : 'Par région'}]
dc_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/5fd5e5d8525afa27210b5c64420d475a531c3b1eb3d6bbd9fed979f3081d', 'KPI' : 'Retours à domicile', 'UPPER_FILTER' : 'Par région'}]
rad_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/a1e16c779ba48901c40ad7daf39dceea2dda8752b6de257b58108a97b9eb', 'KPI' : 'Décès à l’hôpital', 'UPPER_FILTER' : 'Par région'}]
reat_reg = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/ec820131eb62001106cb3f0a74f2adbacefb68f5d11854be593972e4f55f', 'KPI' : 'Tensions hospitalières', 'UPPER_FILTER' : 'Par région'}]
        
list_kpis = hosp_dep + rea_dep + dc_dep + rad_dep + hosp_reg+ rea_reg + dc_reg + rad_reg + reat_dep + reat_reg
domain = pd.DataFrame(list_kpis)
domain.loc[:,'VALUE'] = """<iframe src='"""+ domain['URL'] +"""' frameborder='0' style='overflow:hidden;height:85vh;width:100%' height='100%' width='100%'></iframe> <style>.modebar{display: none!important;}</style>"""
if use_mongo:
    bob.mongo.save_df(domain,'110',db_app,True)
domain

Dataframe 110 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.01493692398071289 secnds ---


,URL,KPI,UPPER_FILTER,VALUE
0,https://api.dev.cashstory.com/api/v1/notebook/...,Hospitalisations,Par département,<iframe src='https://api.dev.cashstory.com/api...
1,https://api.dev.cashstory.com/api/v1/notebook/...,En réanimation,Par département,<iframe src='https://api.dev.cashstory.com/api...
2,https://api.dev.cashstory.com/api/v1/notebook/...,Retours à domicile,Par département,<iframe src='https://api.dev.cashstory.com/api...
3,https://api.dev.cashstory.com/api/v1/notebook/...,Décès à l’hôpital,Par département,<iframe src='https://api.dev.cashstory.com/api...
4,https://api.dev.cashstory.com/api/v1/notebook/...,Hospitalisations,Par région,<iframe src='https://api.dev.cashstory.com/api...
5,https://api.dev.cashstory.com/api/v1/notebook/...,En réanimation,Par région,<iframe src='https://api.dev.cashstory.com/api...
6,https://api.dev.cashstory.com/api/v1/notebook/...,Retours à domicile,Par région,<iframe src='https://api.dev.cashstory.com/api...
7,https://api.dev.cashstory.com/api/v1/notebook/...,Décès à l’hôpital,Par région,<iframe src='https://api.dev.cashstory.com/api...
8,https://api.dev.cashstory.com/api/v1/notebook/...,Tensions hospitalières,Par département,<iframe src='https://api.dev.cashstory.com/api...
9,https://api.dev.cashstory.com/api/v1/notebook/...,Tensions hospitalières,Par région,<iframe src='https://api.dev.cashstory.com/api...
